# Validation: Neuropeptide Ligand/Receptor Expression by Region

This notebook validates expression profiles against known neuroscience facts from `hypothalamic_nuclei_genes.csv`.

**Scoring**: 1 point for each gene where the expected region ranks in the **top 5** for expression.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load data
expr_df = pd.read_parquet('../data/processed/mouse_abc/neuropeptide_expression.parquet')
cell_metadata = pd.read_parquet('../data/processed/mouse_abc/cell_metadata.parquet')

# Join expression with region, filter to neurons only
expr_with_region = expr_df.join(
    cell_metadata.set_index('cell_id')[['region', 'cluster', 'class']], 
    how='inner'
)
NON_NEURONAL = ['30 Astro-Epen', '31 OPC-Oligo', '33 Vascular', '34 Immune']
expr_with_region = expr_with_region[~expr_with_region['class'].isin(NON_NEURONAL)]

# Load expected mappings
expected_df = pd.read_csv('../data/generated/mouse_common/hypothalamic_nuclei_genes.csv')

# Load ligand-receptor list to know what's tracked
lr_df = pd.read_csv('../data/processed/mouse_common/ligand_receptor_mouse.csv')
tracked_genes = set(lr_df['gene_ligand'].unique()) | set(lr_df['gene_receptor'].unique())

# Available genes in expression data
available_genes = set(expr_df.columns)

# Expression threshold
EXPRESSION_THRESHOLD = 3.0

print(f"Neurons with expression data: {len(expr_with_region):,}")
print(f"Tracked genes in LR list: {len(tracked_genes)}")
print(f"Available in expression data: {len(available_genes)}")

In [ ]:
def get_region_ranks(gene: str) -> dict:
    """Get rank of each region for a gene (1 = highest expression)."""
    if gene not in expr_with_region.columns:
        return {}
    
    stats = expr_with_region.groupby('region').agg(
        total_cells=('region', 'count'),
        expressing_cells=(gene, lambda x: (x > EXPRESSION_THRESHOLD).sum())
    )
    stats['pct_expressing'] = 100 * stats['expressing_cells'] / stats['total_cells']
    stats = stats.sort_values('pct_expressing', ascending=False)
    stats['rank'] = range(1, len(stats) + 1)
    
    return stats['rank'].to_dict()

In [ ]:
# Score each expected gene-region pair
results = []

for _, row in expected_df.iterrows():
    area = row['area']
    gene_field = row['gene']
    gene_type = row['type']
    
    # Handle multi-gene entries like "Npy1r/Npy5r" or "Gabra1/Gabrb2/Gabrg2"
    genes = [g.strip() for g in gene_field.split('/')]
    
    for gene in genes:
        # Skip GABA receptors
        if gene.startswith('Gabr') or gene.startswith('Grin'):
            continue
        
        # Skip if not in tracked genes
        if gene not in tracked_genes:
            continue
            
        # Skip if not in available expression data
        if gene not in available_genes:
            continue
        
        # Get ranks
        ranks = get_region_ranks(gene)
        if not ranks:
            continue
        
        rank = ranks.get(area, 999)
        in_top5 = rank <= 5
        score = 1 if in_top5 else 0
        
        results.append({
            'area': area,
            'gene': gene,
            'type': gene_type,
            'expected_rank': rank,
            'in_top5': in_top5,
            'score': score
        })

results_df = pd.DataFrame(results)
print(f"Testable gene-region pairs: {len(results_df)}")
print(f"\nBreakdown by area:")
print(results_df.groupby('area').size())

In [ ]:
# Compute grade per area
area_grades = results_df.groupby('area').agg(
    total_tests=('score', 'count'),
    passed=('score', 'sum'),
).reset_index()
area_grades['grade_pct'] = 100 * area_grades['passed'] / area_grades['total_tests']
area_grades['grade'] = area_grades['grade_pct'].apply(
    lambda x: 'A' if x >= 80 else 'B' if x >= 60 else 'C' if x >= 40 else 'D' if x >= 20 else 'F'
)

print("=== VALIDATION GRADES BY AREA ===\n")
for _, row in area_grades.sort_values('grade_pct', ascending=False).iterrows():
    print(f"{row['area']}: {row['passed']}/{row['total_tests']} ({row['grade_pct']:.0f}%) - Grade {row['grade']}")

print(f"\n=== OVERALL ===")
total = results_df['score'].sum()
possible = len(results_df)
overall_pct = 100 * total / possible
print(f"Total: {total}/{possible} ({overall_pct:.0f}%)")

In [ ]:
# Show details: what passed and what failed
print("=== PASSED (in top 5) ===\n")
passed = results_df[results_df['in_top5']].sort_values(['area', 'expected_rank'])
for area in passed['area'].unique():
    area_passed = passed[passed['area'] == area]
    genes = [f"{r['gene']}(#{r['expected_rank']})" for _, r in area_passed.iterrows()]
    print(f"{area}: {', '.join(genes)}")

print("\n=== FAILED (not in top 5) ===\n")
failed = results_df[~results_df['in_top5']].sort_values(['area', 'expected_rank'])
for area in failed['area'].unique():
    area_failed = failed[failed['area'] == area]
    genes = [f"{r['gene']}(#{r['expected_rank']})" for _, r in area_failed.iterrows()]
    print(f"{area}: {', '.join(genes)}")

---
## Interactive Ranked Heatmaps

Visualize expression rankings across all regions.

In [ ]:
# Get all regions and prepare data for heatmaps
region_counts = expr_with_region['region'].value_counts()
top_regions = region_counts.index.tolist()

# Get cluster profiles for gene classification
cluster_profiles = pd.read_parquet('../data/processed/mouse_abc/cluster_ligand_receptor_profile.parquet')
gene_info = cluster_profiles.groupby('gene')[['is_ligand', 'is_receptor']].first()

# Get ligand and receptor genes
ligand_genes = sorted([g for g in available_genes if g in gene_info.index and gene_info.loc[g, 'is_ligand']])
receptor_genes = sorted([g for g in available_genes if g in gene_info.index and gene_info.loc[g, 'is_receptor']])

print(f"Ligands: {len(ligand_genes)}")
print(f"Receptors: {len(receptor_genes)}")

In [ ]:
# Ranked LIGAND heatmap
rank_data_lig = []
for gene in ligand_genes:
    ranks = get_region_ranks(gene)
    for region in top_regions:
        rank = ranks.get(region, len(top_regions) + 1)
        rank_data_lig.append({'gene': gene, 'region': region, 'rank': rank})

rank_df_lig = pd.DataFrame(rank_data_lig)
rank_pivot_lig = rank_df_lig.pivot(index='gene', columns='region', values='rank').loc[ligand_genes]

# Text annotations (top 5 only)
text_lig = [[f"#{int(rank_pivot_lig.loc[g, r])}" if rank_pivot_lig.loc[g, r] <= 5 else "" 
             for r in rank_pivot_lig.columns] for g in ligand_genes]

# Hover text
hover_lig = [[f"Ligand: {g}<br>Region: {r}<br>Rank: #{int(rank_pivot_lig.loc[g, r])}" 
              for r in rank_pivot_lig.columns] for g in ligand_genes]

# Color (inverted rank)
color_lig = 21 - np.clip(rank_pivot_lig.values, 1, 20)

fig_lig = go.Figure(data=go.Heatmap(
    z=color_lig, x=rank_pivot_lig.columns.tolist(), y=ligand_genes,
    text=text_lig, texttemplate="%{text}", textfont={"size": 9},
    hovertext=hover_lig, hoverinfo="text",
    colorscale='YlOrRd',
    colorbar=dict(title='Rank', tickvals=[1, 5, 10, 15, 20], ticktext=['#20', '#15', '#10', '#5', '#1']),
    zmin=1, zmax=20
))
fig_lig.update_layout(
    title='LIGAND Expression by Region (RANKED)<br><sup>Brighter = higher rank</sup>',
    xaxis_title='Region', yaxis_title='Ligand',
    height=max(500, len(ligand_genes) * 20), width=1100,
    yaxis=dict(tickfont=dict(size=10), autorange='reversed'),
    xaxis=dict(tickangle=45, tickfont=dict(size=9))
)
fig_lig.show()

In [ ]:
# Ranked RECEPTOR heatmap
rank_data_rec = []
for gene in receptor_genes:
    ranks = get_region_ranks(gene)
    for region in top_regions:
        rank = ranks.get(region, len(top_regions) + 1)
        rank_data_rec.append({'gene': gene, 'region': region, 'rank': rank})

rank_df_rec = pd.DataFrame(rank_data_rec)
rank_pivot_rec = rank_df_rec.pivot(index='gene', columns='region', values='rank').loc[receptor_genes]

# Text annotations (top 5 only)
text_rec = [[f"#{int(rank_pivot_rec.loc[g, r])}" if rank_pivot_rec.loc[g, r] <= 5 else "" 
             for r in rank_pivot_rec.columns] for g in receptor_genes]

# Hover text
hover_rec = [[f"Receptor: {g}<br>Region: {r}<br>Rank: #{int(rank_pivot_rec.loc[g, r])}" 
              for r in rank_pivot_rec.columns] for g in receptor_genes]

# Color (inverted rank)
color_rec = 21 - np.clip(rank_pivot_rec.values, 1, 20)

fig_rec = go.Figure(data=go.Heatmap(
    z=color_rec, x=rank_pivot_rec.columns.tolist(), y=receptor_genes,
    text=text_rec, texttemplate="%{text}", textfont={"size": 9},
    hovertext=hover_rec, hoverinfo="text",
    colorscale='YlGnBu',
    colorbar=dict(title='Rank', tickvals=[1, 5, 10, 15, 20], ticktext=['#20', '#15', '#10', '#5', '#1']),
    zmin=1, zmax=20
))
fig_rec.update_layout(
    title='RECEPTOR Expression by Region (RANKED)<br><sup>Brighter = higher rank</sup>',
    xaxis_title='Region', yaxis_title='Receptor',
    height=max(500, len(receptor_genes) * 18), width=1100,
    yaxis=dict(tickfont=dict(size=10), autorange='reversed'),
    xaxis=dict(tickangle=45, tickfont=dict(size=9))
)
fig_rec.show()